# Sprint 深層学習スクラッチ リカレントニューラルネットワーク
# リカレントニューラルネットワークスクラッチ
**リカレントニューラルネットワーク（RNN）** のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

# 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。
$$a_t=x_t・W_x+h_{t-1}・W_h+B$$
$$h_t=tanh(a_t)$$
$a_t$：時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)                     
$h_t$：時刻tの状態・出力 (batch_size, n_nodes)            
$x_t$：時刻tの入力 (batch_size, n_features)            
$W_x$：入力に対する重み (n_features, n_nodes)           
$h_{t-1}$： 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)               
$W_h$：状態に対する重み。 (n_nodes, n_nodes)           
$B$：バイアス項 (n_nodes,)                 
                             
初期状態$h_0$は全て0とすることが多いですが、任意の値を与えることも可能です。               
上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力$x$は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。


# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。                        
フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

In [9]:
import numpy as np
np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [10]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
#batch_size = x.shape[0] # 1  ここでいうbatch_sizeとは，シークエンスの数である
#n_sequences = x.shape[1] # 3　ここでいうn_sequencesとは，シークエンスの長さ，要素の個数である
#n_features = x.shape[2] # 2
#n_nodes = w_x.shape[1] # 4
h = np.zeros((1,4)) # (batch_size, n_nodes) ｈの初期値
#h_t = np.zeros((batch_size,n_sequences,n_nodes)) #ｈの結果を入れる(batch_size,n_sequences,n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [12]:
class SimpleRNN:
    def __init__(self, w_x, w_h, h, b):
        """
        Paramator
        ---------
        w_x : 入力に対する重み (n_features, n_nodes)
        w_h : 状態に対する重み。 (n_nodes, n_nodes) 
        h   : 状態の初期値(batch_size, n_nodes)
        b   : バイアス(n_nodes,) 
        """
        self.w_x = w_x #(n_features, n_nodes)
        self.w_h = w_h #(n_nodes, n_nodes)
        self.h = h
        self.b = b     #(n_nodes,)
    
    def _tanh(self,a_t):
        """活性化関数：ハイパボリックタンジェント"""
        return np.tanh(a_t)
    
    
    def _forward(self,x):
        """
        RNNのフォワードプロパゲーション
        input
        ------
        x   : 入力． (batch_size, n_sequences, n_features)
        
        output
        -------
        h_t : 時刻tの状態・出力 (batch_size, n_sequences, n_nodes) 
        """
        batch_size = x.shape[0] # 1
        n_sequences = x.shape[1] # 3
        n_features = x.shape[2] # 2
        n_nodes = self.w_x.shape[1] # 4
        
        h_t = np.zeros((batch_size,n_sequences,n_nodes)) #ｈの結果を入れる(batch_size,n_sequences,n_nodes)
        
        for i in range(n_sequences):
            if np.all(h_t):
                a_0 = x[:,i,:]@self.w_x + self.h@self.w_h + self.b
                h_t[:,i,:] = self._tanh(a_0)
                break
            # ２回目以降のループ
            else:
                a_t = x[:,i,:]@w_x + h_t[:,i-1,:]@self.w_h + self.b
                h_t[:,i,:] = self._tanh(a_t)
        
        return h_t

In [14]:
rnn = SimpleRNN(w_x, w_h, h, b)
h_t = rnn._forward(x)
h_t

array([[[0.76188798, 0.76213958, 0.76239095, 0.76255841],
        [0.792209  , 0.8141834 , 0.83404912, 0.84977719],
        [0.79494228, 0.81839002, 0.83939649, 0.85584174]]])